#### Check De Data

In [66]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
import dvc.api
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_hastie_10_2
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from IPython.display import display

import logging

In [67]:
path="../data/prepared/beer_profile_and_ratings.csv"

In [68]:
with open(path,'rb') as f:
    df = pd.read_csv(f)
    display(df.head())
    print(df.columns)
    
    df_preprocess = df[['Name', 'Style', 'ABV',
       'Min IBU', 'Max IBU', 'Astringency', 'Body', 'Alcohol', 'Bitter',
       'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty',
       'review_aroma', 'review_appearance', 'review_palate', 'review_taste',
       'review_overall']]
    df_preprocess.set_index('Name')
    df_preprocess.to_csv("../data/prepared/beer_profile_and_ratings.csv", index=False)
    display(df_preprocess.head())
    
    

,Name,Style,Brewery,Beer Name (Full),Description,ABV,Min IBU,Max IBU,Astringency,Body,...,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews
0,Amber,Altbier,Alaskan Brewing Co.,Alaskan Brewing Co. Alaskan Amber,"Notes:Richly malty and long on the palate, wit...",5.3,25,50,13,32,...,33,57,8,111,3.498994,3.636821,3.556338,3.643863,3.847082,497
1,Double Bag,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Double Bag,"Notes:This malty, full-bodied double alt is al...",7.2,25,50,12,57,...,24,35,12,84,3.798337,3.846154,3.904366,4.024948,4.034304,481
2,Long Trail Ale,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Long Trail Ale,Notes:Long Trail Ale is a full-bodied amber al...,5.0,25,50,14,37,...,10,54,4,62,3.409814,3.667109,3.600796,3.631300,3.830239,377
3,Doppelsticke,Altbier,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Notes:,8.5,25,50,13,55,...,49,40,16,119,4.148098,4.033967,4.150815,4.205163,4.005435,368
4,Sleigh'r Dark Doüble Alt Ale,Altbier,Ninkasi Brewing Company,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Notes:Called 'Dark Double Alt' on the label.Se...,7.2,25,50,25,51,...,11,51,20,95,3.625000,3.973958,3.734375,3.765625,3.817708,96


Index(['Name', 'Style', 'Brewery', 'Beer Name (Full)', 'Description', 'ABV',
       'Min IBU', 'Max IBU', 'Astringency', 'Body', 'Alcohol', 'Bitter',
       'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty',
       'review_aroma', 'review_appearance', 'review_palate', 'review_taste',
       'review_overall', 'number_of_reviews'],
      dtype='object')


,Brewery,Style,ABV,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,...,Salty,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall
0,Alaskan Brewing Co.,Altbier,5.3,25,50,13,32,9,47,74,...,0,33,57,8,111,3.498994,3.636821,3.556338,3.643863,3.847082
1,Long Trail Brewing Co.,Altbier,7.2,25,50,12,57,18,33,55,...,0,24,35,12,84,3.798337,3.846154,3.904366,4.024948,4.034304
2,Long Trail Brewing Co.,Altbier,5.0,25,50,14,37,6,42,43,...,0,10,54,4,62,3.409814,3.667109,3.600796,3.631300,3.830239
3,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Altbier,8.5,25,50,13,55,31,47,101,...,1,49,40,16,119,4.148098,4.033967,4.150815,4.205163,4.005435
4,Ninkasi Brewing Company,Altbier,7.2,25,50,25,51,26,44,45,...,1,11,51,20,95,3.625000,3.973958,3.734375,3.765625,3.817708


array([ 1., -1.,  1., ..., -1., -1.,  1.])

#### Check data thruough dvc pipe

In [ ]:
path="data/prepared/beer_profile_and_ratings.csv"
repo="https://github.com/stijnhering/PreTraineeship"
version="<GIT COMMIT>"
remote="storage"

In [ ]:
data_url = dvc.api.get_url(path=path, repo=repo)
print(data_url)

In [ ]:
with dvc.api.open(path, 
                  repo=repo) as fd:
        df = pd.read_csv(fd)
        display(df)
    